In [2]:
import string

In [3]:
# utility

In [4]:
def remove_punctuation(s):
    table = str.maketrans('','',string.punctuation)
    return s.translate(table)

In [12]:
i = 0
for line in open('data/robert_frost.txt','r', encoding='utf-8'):
    l = line.strip().lower()
    ret = remove_punctuation(l).split()
    print(ret)
    i += 1
    if i>5:
        break

['two', 'roads', 'diverged', 'in', 'a', 'yellow', 'wood']
['and', 'sorry', 'i', 'could', 'not', 'travel', 'both']
['and', 'be', 'one', 'traveler', 'long', 'i', 'stood']
['and', 'looked', 'down', 'one', 'as', 'far', 'as', 'i', 'could']
['to', 'where', 'it', 'bent', 'in', 'the', 'undergrowth']
[]


In [22]:
def get_robert():
    word2ind = {"start":0,"end":1}
    current_index = 2
    sentences = []
    for line in open('data/robert_frost.txt','r', encoding='utf-8'):
        if line:
            l = line.strip().lower()
            words = remove_punctuation(l).split()
            sentence = []
            for word in words:
                if word not in word2ind:
                    word2ind[word] = current_index
                    current_index += 1
                idx = word2ind[word]
                sentence.append(idx)
            
        sentences.append(sentence)
    return sentences,word2ind
    

In [31]:
def init_weight(Mi, Mo):
    return np.random.randn(Mi, Mo) / np.sqrt(Mi + Mo)

In [18]:
# poetry

In [20]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [21]:
class SimpleRNN(object):
    def __init__(self,D,M,V,f,session):
        self.D = D # dimension of embedding 
        self.M = M # dimension of hidden layer
        self.V = V # dimension of vocabulary
        self.f = f #activation function
        self.session = session
        
    def build(self, we,wx,wh,wo,h0,bh,bo):
        self.we = tf.Variable(we)
        self.wx = tf.Variable(wx)
        self.wh = tf.Variable(wh)
        self.wo = tf.Variable(wo)
        self.h0 = tf.Variable(h0)
        self.bh = tf.Variable(bh)
        self.bo = tf.Variable(bo)
        self.params = [self.we,self.wx,self.wh,self.wo,self.h0,self.bh,self.bo]
        
        #ez access
        V = self.V
        D = self.D
        M = self.M
        
        #placeholder
        self.tfX = tf.placeholder(X,shape = (None,),'X')
        self.tfY = tf.placeholder(Y,shape = (None,),'Y')
        
        XW = tf.nn.embedding_lookup(we,self.tfX)
        
        XW_WX = tf.matmul(XW,self.wx)
        
        def recurrent(h,xw_wx_t):
            h_t = tf.reshape(h,(M,1))
            h_new = self.f(xw_wx_t + tf.matmul(h_t,self.wh) + self.bh)
            res = tf.reshape(h_new,(M,1))
            return res
        
        h = tf.scan(
            fn = recurrent,
            elems = XW_WX,
            initializer = self.h0
        )
        
        #output
        logit = tf.matmul(h, self.wo) + self.bo
        prediction = tf.argmax(logit,axis = 1)
        self.output_prob = tf.nn.softmax(logit)
        
        cost_weight = tf.transpose(self.wo,[1,0])
        h = tf.reshape(self.h,(-1,M))
        labels = tf.reshape(self.tfY,(-1,1))
        
        self.cost = tf.reduce_mean(
            tf.nn.sampled_softmax_loss(
                weights = ,
                biases = self.b0,
                labels = labels ,
                inputs = h,
                num_sampled = 50,
                num_classes = V
            )
        )
        
        self.predict_op = prediction
        self.train_op = tf.train.AdadeltaOptimizer(1e-2).minimize(self.cost)
        #init variables
        init = tf.global_variables_initializer()
        self.session.run(init)
    
    
    def fit(self,X,epi = 500, show_figure = False):
        N = len(X)
        V = self.V
        M = self.M
        D = self.D
        
        #init weights
        we = init_weight(np.random.rand(V,D)).astype(np.float32)
        wx = init_weight(np.random.rand(D,M)).astype(np.float32)
        wh = init_weight(np.random.rand(M,M)).astype(np.float32)
        wo = init_weight(np.random.rand(M,K)).astype(np.float32)
        
        bh = init_weight(np.zeros(M)).astype(np.float32)
        bo = init_weight(np.zeros(K)).astype(np.float32)
        h0 = init_weight(np.zeros(M)).astype(np.float32)
        
        self.build(we,wx,wh,wo,h0,bh,bo)
        
        costs = []
        n_total = sum((len(sentence) + 1) for sentence in X)
        
        for i in range(epi):
            X = shuffle(X)
            n_correct = 0
            cost = 0
            
            for j in range(N):
                
        
        
    
    def predict(self):
        pass
    
    def generate(self):
        pass
    
    def save(self):
        pass
    
    @staticmethod
    def load(self):
        pass

In [28]:
def train_poety(session,dim,savefile):
    sentences,word2ind = get_robert()
    rnn = SimpleRNN(dim,dim,len(word2ind),session)
    rnn.fit(sentences,epi = 20,show_figure = False)
    rnn.save(savefile)

In [29]:
def gemerate_poety(session,savefile):
    sentences, word2ind = get_robert()
    rnn = SimpleRNN.load(savefile,tf.nn.relu,savefile)
    #get the initial word distribution
    V = len(sentences)
    pi = np.zeros(V)
    for s in sentences:
        pi[s[0]] += 1
    pi = pi/np.sum(pi)
    
    return rnn.generate(pi,word2ind)

In [30]:
dim = 50
savefile = 'rnn_poety.npz'
session = tf.InteractiveSession()
train_poety(session,dim,savefile)
generate_poety(session,savefile)

D:\anaconda\lib\site-packages\tensorflow\python\client\session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TypeError: __init__() missing 1 required positional argument: 'session'